<a href="https://colab.research.google.com/github/audwls4545/metaverse/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#한글 분석을 위한 konlpy 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 40.4 MB/s eta 0:00:00


In [4]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/colab_data_file/ratings_train.txt', sep = '\t')
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [6]:
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [7]:
# Null인 곳에 ' '(공백) 체우기 -> 결측치 데이터 처리
train_df = train_df.fillna(' ')

In [9]:
#re.sub 예시
import re
re.sub(r'[^ㄱ-ㅣ가-힣\s]','','나는 대한민국 123 국민 abc')

'나는 대한민국  국민 '

In [10]:
# 이상치 제거 -> 한글이 아닌 모든 것(ex. 이모티콘, 숫자, 영어 등)
# 정규표현식을 사용해서 한글만 남기기
# 맞춤법 api와 띄어쓰기 api 사용하면 더 성능이 좋음
import re

result = []

for temp in train_df['document']:
    kor_str = re.sub(r'[^ㄱ-ㅣ가-힣\s]','',temp)
    result.append(kor_str)

train_df['document'] = result

In [12]:
# ..., ? 와 같은 문자들 삭제 확인
train_df

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [13]:
# id 컬럼 필요 없으니 삭제 -> 같이 학습하면 성능이 다운될 수 있음
train_df.drop('id', axis = 1, inplace = True)

In [14]:
train_df

,document,label
0,아 더빙 진짜 짜증나네요 목소리,0
1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...
149995,인간이 문제지 소는 뭔죄인가,0
149996,평점이 너무 낮아서,1
149997,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [16]:
# Okt 형태소 분석
from konlpy.tag import Okt

okt = Okt()

okt.nouns(train_df['document'][0]) #명사만 추출
okt.morphs(train_df['document'][0]) #모든 형태소 추출

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [17]:
# 형태소 추출 함수화
def okt_tokenizer(text):
    return okt.morphs(text)

In [18]:
# 단어들 가중치 부여
from sklearn.feature_extraction.text import TfidfVectorizer

#min_df = 3 -> 3번 미만으로 나온 단어는 제거해라
#max_df = 0.9 -> 전체 문장에서 90%이상으로 나오는 단어 제거해라
tfidf_vect = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range = (1, 2), min_df = 3, max_df = 0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
import pickle

pickle.dump(tfidf_vect, open('tfidf.pickle', 'wb'))
pickle.dump(tfidf_matrix, open('tfidf_matrix.pickle', 'wb'))

In [ ]:
#코랩에서 내 컴퓨터로 저장하는 코드 추가 필요

In [20]:
r = tfidf_vect.transform(['정말 재미있는 영화네요'])

r.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [21]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(tfidf_matrix, train_df['label'])

LogisticRegression()

In [22]:
reply = tfidf_vect.transform(['아 더빙 진짜 짜증나네 목소리'])
model.predict(reply) #부정적인 글이다 -> 0

array([0])

In [24]:
reply = tfidf_vect.transform(['영화 진짜 재미있었습니다.'])
model.predict(reply) #긍정적인 글이다 -> 1

array([1])